# ReadMe
- This notebook is written by David Sanotona as part of OnlinePajak technical test
- Key assumptions:
    - 8 months data used as holdout, 4 months for demonstration -> this to mimic selling the apps that we have existing data already
    - This notebook assume the csv is in same folder as the notebook
    - libraries requirements must be installed according to requirements.txt

# Methodology
The notebook check 3 scores: Vendor operational efficiency, Vendor financial efficiency, and Audit Risk
Scores are then computed against itself and/or against peers:
1. Vendor operational efficiency: detecting slow or inefficient vendor operationally
- Against itself:2x std deviation from company’s mean after data is normalized 
- Against peer:2x std deviation from peers’ mean after data is normalized
2. Vendor financial efficiency: detecting potential phantom vendor/frauds
- Against itself: -
- Against peer: Local Isolation Forest
3. Audit Risk: whether data is consistent with benchmark VAT
- Against itself: deviation more than 10%
- Against peer: -
The 3 scores are then feeded into LLM to summarize and generate report


# Environment Set Up

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from transformers import pipeline
import os
import warnings
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

In [3]:
explainer = pipeline("text2text-generation", model="google/flan-t5-base")

Device set to use cpu


In [4]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
warnings.filterwarnings('ignore')

# Load Data

In [28]:
df=pd.read_csv('company_metrics_data.csv')
df.head()

,company_id,month_key,avg_dpp_per_invoice,monthly_spend_volatility,total_dpp,total_inbound_tax_invoices,total_vat,ammend_rate,approval_rate,friction_value_ratio,hard_failure_rate,top_1_vendor_share,top_5_vendor_share,unique_vendors,vendor_concentration_index,vendor_repeat_rate
0,C001,2025-05-01,41683335,0.067,851630840,169,78054326,0.096,0.847,0.057,0.038,0.674,0.726,55,4134,0.493
1,C001,2025-03-01,19642295,0.301,169521763,207,16600946,0.010,0.791,0.199,0.033,0.458,0.760,79,1910,0.582
2,C001,2025-07-01,16987409,0.078,467365513,46,42194928,0.010,0.531,0.459,0.066,0.506,0.700,91,3811,0.908
3,C001,2025-06-01,72656597,0.174,480381782,68,43816491,0.010,NaN,0.252,0.059,0.280,0.630,72,1258,0.816
4,C001,2025-12-01,25818236,0.635,661033053,182,61417966,0.010,0.601,0.389,0.054,0.230,0.778,36,1351,0.980


# Start Notebook

## 1. Inspect Data

In [29]:
df.dtypes

company_id                     object
month_key                      object
avg_dpp_per_invoice             int64
monthly_spend_volatility      float64
total_dpp                       int64
total_inbound_tax_invoices      int64
total_vat                       int64
ammend_rate                   float64
approval_rate                 float64
friction_value_ratio          float64
hard_failure_rate             float64
top_1_vendor_share            float64
top_5_vendor_share            float64
unique_vendors                  int64
vendor_concentration_index      int64
vendor_repeat_rate            float64
dtype: object

In [30]:
df.isna().sum()

company_id                     0
month_key                      0
avg_dpp_per_invoice            0
monthly_spend_volatility       0
total_dpp                      0
total_inbound_tax_invoices     0
total_vat                      0
ammend_rate                    8
approval_rate                 30
friction_value_ratio           8
hard_failure_rate              0
top_1_vendor_share             0
top_5_vendor_share             0
unique_vendors                 0
vendor_concentration_index     0
vendor_repeat_rate             0
dtype: int64

In [31]:
print(df.company_id.nunique())
print(df.month_key.nunique())

50
12


In [32]:
print(df.month_key.unique())

['2025-05-01' '2025-03-01' '2025-07-01' '2025-06-01' '2025-12-01'
 '2025-02-01' '2025-08-01' '2025-11-01' '2025-04-01' '2025-01-01'
 '2025-10-01' '2025-09-01']


## 2. Modelling

In [33]:
def prepare_data(df):
    ''' 
    1. Data Preparation (8:4 Split)
    2. Data cleaning
    '''
    df_clean = df.sort_values(by=['company_id', 'month_key']).copy()
    
    missing_mask = df_clean.isna()
    df_filled = df_clean.groupby('company_id').ffill()
    for col in df_filled.columns:
        df_clean[col] = df_filled[col]
        
    df_clean['Data_Imputed_Warning'] = missing_mask.any(axis=1)
    
    all_months = sorted(df_clean['month_key'].unique())
    historical_months = all_months[:8]
    deployment_months = all_months[8:]
    
    return df_clean, historical_months, deployment_months

In [ ]:
def score_entity(df_clean, target_entity, target_month, vat_rate_input, historical_months):
    target_row = df_clean[(df_clean['company_id'] == target_entity) & (df_clean['month_key'] == target_month)]
    if target_row.empty: return {"error": "No data found."}
    
    target_data = target_row.iloc[0]
    history_end = historical_months[-1]
    df_history = df_clean[df_clean['month_key'] <= history_end]
    self_history = df_history[df_history['company_id'] == target_entity]

    # 1. Operational Efficiency 
    metric = 'friction_value_ratio'
    current_val = target_data[metric]
    peer_mean = df_history[metric].mean()
    peer_std = df_history[metric].std() + 1e-9
    peer_z = (current_val - peer_mean) / peer_std
    
    if len(self_history) > 1:
        self_mean = self_history[metric].mean()
        self_std = self_history[metric].std() + 1e-9
        self_z = (current_val - self_mean) / self_std
    else:
        self_mean, self_std, self_z = peer_mean, peer_std, peer_z

    flag_op = (self_z > 2.0) or (peer_z > 2.0)

    # 2. Financial Efficiency 
    features_fin = ['monthly_spend_volatility', 'vendor_repeat_rate', 'top_1_vendor_share', 'top_5_vendor_share']
    train_data_fin = df_history.dropna(subset=features_fin)
    
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    if len(train_data_fin) > 10: 
        iso_forest.fit(train_data_fin[features_fin])
        anomaly_score = iso_forest.predict(pd.DataFrame([target_data[features_fin]]))[0]
        flag_phantom = (anomaly_score == -1)
    else:
        flag_phantom = False

    peer_fin_means = train_data_fin[features_fin].mean().to_dict()

    # 3. Audit Risk
    expected_vat = target_data['total_dpp'] * (vat_rate_input / 100)
    vat_dev = np.abs(target_data['total_vat'] - expected_vat) / (expected_vat + 1e-9)
    flag_audit = vat_dev > 0.10

    return {
        "Entity": target_entity, "Month": target_month,
        "Op_Eff": {
            "Current": current_val, 
            "Self_Mean": self_mean, 
            "Peer_Mean": peer_mean, 
            "Self_Threshold": self_mean + (2 * self_std), 
            "Peer_Threshold": peer_mean + (2 * peer_std), 
            "Flagged": flag_op
        },
        "Fin_Eff": {"Current_Features": target_data[features_fin].to_dict(), "Peer_Means": peer_fin_means, "Flagged": flag_phantom},
        "Audit": {"Deviation": vat_dev, "Flagged": flag_audit}
    }

In [52]:
_AI_MODEL = None 

def load_ai_model():
    global _AI_MODEL
    if _AI_MODEL is None:
        print("Loading AI Model (flan-t5-base)...")
        from transformers import pipeline
        _AI_MODEL = pipeline("text2text-generation", model="google/flan-t5-base")
        print("AI Model ready!")
    return _AI_MODEL

def generate_cfo_report(results, use_llm=True):
    if "error" in results: return results["error"]
    
    op = results["Op_Eff"]
    fin = results["Fin_Eff"]
    aud = results["Audit"]

    # 1. Deterministic Table
    report = f"EXECUTIVE DATA SUMMARY | ENTITY: {results['Entity']} | AS-OF: {results['Month']}\n"
    report += "="*75 + "\n"
    report += f"1. VENDOR OPERATIONAL EFFICIENCY: {'ALERT - INEFFICIENT' if op['Flagged'] else 'STABLE'}\n"
    report += f"   - Current Friction: {op['Current']:.2%}\n"
    report += f"   - Baseline: Company Avg {op['Self_Mean']:.2%} | Peer Avg {op['Peer_Mean']:.2%}\n"
    report += f"   - Danger Threshold: {op['Self_Threshold']:.2%}\n"

    report += f"\n2. VENDOR FINANCIAL EFFICIENCY: {'ALERT - ANOMALY' if fin['Flagged'] else 'NORMAL'}\n"
    report += f"   - Top 1 Vendor Share: {fin['Current_Features']['top_1_vendor_share']:.2%} (Peer Avg: {fin['Peer_Means']['top_1_vendor_share']:.2%})\n"
    report += f"   - Status: {'High-risk spend volatility detected' if fin['Flagged'] else 'Standard spend behavior'}\n"

    report += f"\n3. EXTERNAL AUDIT RISK: {'ALERT - HIGH RISK' if aud['Flagged'] else 'LOW RISK'}\n"
    report += f"   - VAT Deviation: {aud['Deviation']:.2%} (Limit: 10.00%)\n"

    # 2. Categorical AI Advisor
    ai_advice = "\nAI STRATEGIC ADVICE:\n" + "-"*75 + "\n"
    
    if not (op['Flagged'] or fin['Flagged'] or aud['Flagged']):
        ai_advice += "All indicators are within safe parameters. No immediate action required. Maintain routine oversight.\n"
    else:
        actions = []
        if aud['Flagged']: actions.append("reconcile tax accounts and pause filings")
        if op['Flagged']:  actions.append("conduct a vendor performance audit")
        if fin['Flagged']: actions.append("investigate spend anomalies for potential fraud")
        
        action_str = ", ".join(actions)
        
        prompt = f"Transform this into a professional 1-sentence CFO recommendation: 'The CFO should {action_str}.'"

        try:
            generator = load_ai_model()
            output = generator(prompt, max_new_tokens=45, do_sample=False)
            ai_advice += "Based on detected risks, the CFO should " + output[0]['generated_text'].replace("The CFO should", "").strip() + "\n"
        except:
            ai_advice += f"Immediate Action Required: The CFO should {action_str}.\n"

    # 3. Disclaimer
    disclaimer = "\n" + "="*75 + "\n"
    disclaimer += "DISCLAIMER: This is a recommendation based on statistical data. A more thorough investigation is recommended before taking action."

    return report + ai_advice + disclaimer

## 3. Product Output

In [ ]:
df_clean, historical_months, deployment_months = prepare_data(df)

entity_dd = widgets.Dropdown(options=df_clean['company_id'].unique(), description='Company ID:')
month_dd = widgets.Dropdown(options=deployment_months, description='Live Month:')
vat_input = widgets.FloatText(value=11.0, description='Expected VAT %:', style={'description_width': 'initial'})
run_btn = widgets.Button(description='Run Intelligence', button_style='primary')
output_box = widgets.Output()

def on_click(b):
    with output_box:
        clear_output(wait=True)
        print("Running analysis...")
        results = score_entity(df_clean, entity_dd.value, month_dd.value, vat_input.value, historical_months)
        report_text = generate_cfo_report(results, use_llm=True)
        
        display(HTML(f"""
        <div style="border: 1px solid #333; padding: 20px; border-radius: 4px; background: #ffffff; font-family: 'Courier New', Courier, monospace;">
            <pre style="white-space: pre-wrap; font-size: 14px; color: #111;">{report_text}</pre>
        </div>
        """))

run_btn.on_click(on_click)
display(widgets.HBox([entity_dd, month_dd, vat_input]), run_btn, output_box)

Button(button_style='primary', description='Run Intelligence', style=ButtonStyle())

Output()